# Wprowadzenie do ipywidgets

In [19]:
import datetime
import numpy as np
import pandas as pd

from ipywidgets import widgets
import plotly.graph_objects as go

## 1. Podstawowy przykład z kontrolka typu suwak

In [2]:
# na początek kilka przykładowych kontrolek i sposobów manipulacji wartościami niektórych ich parametrów
# po tej deklaracji widget domyślnie powinien się wyświetlić
widgets.IntSlider(
    value=7, # aktualna wartość slidera
    min=0,
    max=10,
    step=1, # krok zmiany wartości slidera
    description='Test:', # opis slidera - etykieta
    disabled=False, # możemy w razie potrzeby uniemożliwić zmianę wartości na tym widgecie
    continuous_update=False,
    orientation='horizontal', # slidery mogą być również zorientowane pionowo
    readout=True, # czy pokazywać wartość slidera obok kontrolki
    readout_format='d' # format zapisu wartości slidera obok kontrolki
)

IntSlider(value=7, continuous_update=False, description='Test:', max=10)

In [3]:
# powyższy sposób deklaracji ma jednak wadę, gdyż nie możemy (na razie) odwołać się do obiektu, który go reprezentuje
# przypiszmy więc widget do zmiennej i odczytajmy jakieś wartości po zmianie
wiek = widgets.IntSlider(
    value=7, # aktualna wartość slidera
    min=0,
    max=120,
    step=1, # krok zmiany wartości slidera
    description='Test:', # opis slidera - etykieta
    disabled=False, # możemy w razie potrzeby uniemożliwić zmianę wartości na tym widgecie
    continuous_update=False,
    orientation='horizontal', # slidery mogą być również zorientowane pionowo
    readout=True, # czy pokazywać wartość slidera obok kontrolki
    readout_format='d' # format zapisu wartości slidera obok kontrolki
)
wiek
# korzystając z przeglądarki może być konieczne wywołanie metody display, aby widget był widoczny
# display(wiek)

IntSlider(value=7, continuous_update=False, description='Test:', max=120)

In [4]:
# odczytujemy aktualną wartość slidera
wiek.value

7

## 2. Nasłuchiwanie zdarzeń kontrolek

Nietrudno się domyślić, że pozwala to na dwustronną interakcję - odczytujemy jakieś dane, na podstawie danych deklarujemy początkowe wartości widgetu, a następnie poprzez manipulacje kontrolką możemy sczytać nowe wartości i na ich podstawie wykonać jakąś akcję. W zależności od kontrolek zgłaszane są różne eventy, których wystąpienia możemy nasłuchiwać. Musimy zarejestrować specjalny obiekt obserwatora eventów dla takiej kontrolki poprzez metodę `observe`.

In [5]:
# dokumentacja dla metody observe
print(widgets.Widget.observe.__doc__)

Setup a handler to be called when a trait changes.

        This is used to setup dynamic notifications of trait changes.

        Parameters
        ----------
        handler : callable
            A callable that is called when a trait changes. Its
            signature should be ``handler(change)``, where ``change`` is a
            dictionary. The change dictionary at least holds a 'type' key.
            * ``type``: the type of notification.
            Other keys may be passed depending on the value of 'type'. In the
            case where type is 'change', we also have the following keys:
            * ``owner`` : the HasTraits instance
            * ``old`` : the old value of the modified trait attribute
            * ``new`` : the new value of the modified trait attribute
            * ``name`` : the name of the modified trait attribute.
        names : list, str, All
            If names is All, the handler will apply to all traits.  If a list
            of str, handler wil

In [6]:
# przykład wykorzystania dla poprzedniej kontrolki typu slider
output = widgets.Output()

display(output)

def zmiana_wartosci(change):
    with output:
        print(f"Aktualna wartość slidera: {change['new']}")

# rejestracja obserwatora eventu, tutaj będą przekazywane tylko wartości właściwości values danego widgetu
wiek.observe(zmiana_wartosci, names='value')

Output()

Nie jest to może zbyt efektowny przykład, ale skoro możemy przekazywać wartości z i do kontrolki to już nie stoi na przeszkodzie, żeby budować praktycznie dowolne interakcje pomiędzy frontendem i backendem naszego notatnika.

## 3. Manipulowanie parametrami wykresu poprzez kontrolki

In [29]:
# będzie to prosty wykres słupkowy, który zostanie utworzony na podstawie losowo wygenerowanych wartości z przedziału
# określonego przez kontrolki
# rozpoczniemy od utworzenia kontrolek

# kontrolka zakresu dla generatora liczb losowych
zakres = widgets.IntRangeSlider(
    value=[10, 20],
    min=0,
    max=100,
    step=1,
    description='Zakres liczb losowych:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# kontrolka do wskazania ilości wartości do wylosowania
ilosc = widgets.BoundedIntText(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Ile liczb:',
    disabled=False
)

# wartości do wykresu
values = np.random.randint(low=zakres.min, high=zakres.max, size=ilosc.value)

# kontrolka z wykresem
wykres = go.Histogram(x=values, opacity=0.75, name='Przykładowy wykres.')
g = go.FigureWidget(data=wykres,
                    layout=go.Layout(
                        title=dict(
                            text='Rozkład liczb losowych'
                        ),
                        barmode='overlay'
                    ))

# funkcja aktualizująca
def response(change):
    with g.batch_update():
        values = np.random.randint(low=zakres.min, high=zakres.max, size=ilosc.value)
        g.data[0].x = values
        print(values)

        
# rejestracja nasłuchiwania eventu
zakres.observe(response, names="value")
ilosc.observe(response, names="value")
        
# zorganizujemy to w postaci pudełka poziomego
display(widgets.HBox([zakres, ilosc]))
g


FigureWidget({
    'data': [{'name': 'Przykładowy wykres.',
              'opacity': 0.75,
              'type…

[70 22 15 22 97 99 39 32 92 72]
